<a href="https://colab.research.google.com/github/vinay10949/AnalyticsAndML/blob/master/FeatureEngineering/Categorical-Variable-Encoding/Comparison_categorical_encoding_techniques_Extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install feature_engine
!pip install --user kaggle
!pip install  category_encoders 
!mkdir .kaggle
import json
token = {"username":"vinay10949","key":"59c4901452f76ce62979e5b0997e240b"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)



  Created wheel for numpydoc: filename=numpydoc-0.9.2-cp36-none-any.whl size=31893 sha256=d45cc3fa6da6b5c5bc07d9124f4b61140c21d2f43e007a1bb5f7310faf709080
  Stored in directory: /root/.cache/pip/wheels/96/f3/52/25c8e1f40637661d27feebc61dae16b84c7cdd93b8bc3d7486
Successfully built numpydoc
     |████████████████████████████████| 102kB 2.3MB/s 


In [4]:


!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c titanic
!unzip train.csv.zip



  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 6.87MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 25.4MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 62.2MB/s]
unzip:  cannot find or open train.csv.zip, train.csv.zip.zip or train.csv.zip.ZIP.


## Comparison of Categorical Variable Encodings

Comparison of the performance of the different feature categorical encoding techniques we learned so far.

We will compare:

- One hot encoding
- Replacing labels by the count
- Ordering labels according to target
- Mean Encoding
- WoE
- Backward Difference Coding
- Base N
- Binary
- CatBoost Encoding
- Hashing Encoding
- Helmert Encoding
- James-Stein Encoder
- Leave One Out
- M-estimate
- Polynomial Coding
- Sum Coding

In [0]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

### Data Prep

In [7]:
# let's load the titanic dataset

# we will only use these columns in the demo
cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
        'Sex', 'Cabin', 'Embarked', 'Survived']

data = pd.read_csv('train.csv', usecols=cols)

data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [9]:
# let's check for missing data

data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [0]:
# Drop observations with NA in Fare and embarked

data.dropna(subset=['Fare', 'Embarked'], inplace=True)

In [11]:
# Now we extract the first letter of the cabin

data['Cabin'] = data['Cabin'].astype(str).str[0]

data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,n,S
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,n,S
3,1,1,female,35.0,1,0,53.1000,C,S
4,0,3,male,35.0,0,0,8.0500,n,S


In [0]:
# drop observations with cabin = T, they are too few

data = data[data['Cabin'] != 'T']

In [13]:
# Let's divide into train and test set

X_train, X_test, y_train, y_test = train_test_split(
   data.drop(labels='Survived', axis=1),  # predictors
    data['Survived'],  # target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape


((621, 8), (267, 8))

In [0]:
# Let's replace null values in numerical variables by the mean


def impute_na(df, variable, value):
    df[variable].fillna(value, inplace=True)


impute_na(X_test, 'Age', X_train['Age'].mean())
impute_na(X_train, 'Age',  X_train['Age'].mean())
# note how I impute first the test set, this way the value of
# the median used will be the same for both train and test

In [15]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
352,3,male,15.0,1,1,7.2292,n,C
125,3,male,12.0,1,0,11.2417,n,C
579,3,male,32.0,0,0,7.9250,n,S
424,3,male,18.0,1,1,20.2125,n,S
119,3,female,2.0,4,2,31.2750,n,S


In [16]:
# let's check that we have no missing data after NA imputation

X_train.isnull().sum(), X_test.isnull().sum()

(Pclass      0
 Sex         0
 Age         0
 SibSp       0
 Parch       0
 Fare        0
 Cabin       0
 Embarked    0
 dtype: int64, Pclass      0
 Sex         0
 Age         0
 SibSp       0
 Parch       0
 Fare        0
 Cabin       0
 Embarked    0
 dtype: int64)

### One Hot Encoding

In [17]:
def get_OHE(df):

    df_OHE = pd.concat(
        [df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']],
         pd.get_dummies(df[['Sex', 'Cabin', 'Embarked']], drop_first=True)],
        axis=1)

    return df_OHE


X_train_OHE = get_OHE(X_train)
X_test_OHE = get_OHE(X_test)

X_train_OHE.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_n,Embarked_Q,Embarked_S
352,3,15.0,1,1,7.2292,1,0,0,0,0,0,0,1,0,0
125,3,12.0,1,0,11.2417,1,0,0,0,0,0,0,1,0,0
579,3,32.0,0,0,7.9250,1,0,0,0,0,0,0,1,0,1
424,3,18.0,1,1,20.2125,1,0,0,0,0,0,0,1,0,1
119,3,2.0,4,2,31.2750,0,0,0,0,0,0,0,1,0,1


In [18]:
X_test_OHE.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_n,Embarked_Q,Embarked_S
14,3,14.000000,0,0,7.8542,0,0,0,0,0,0,0,1,0,1
159,3,29.696519,8,2,69.5500,1,0,0,0,0,0,0,1,0,1
764,3,16.000000,0,0,7.7750,1,0,0,0,0,0,0,1,0,1
742,1,21.000000,2,2,262.3750,0,1,0,0,0,0,0,0,0,0
484,1,25.000000,1,0,91.0792,1,1,0,0,0,0,0,0,0,0


### Count encoding

In [19]:
def categorical_to_counts(df_train, df_test):

    # make a temporary copy of the original dataframes
    df_train_temp = df_train.copy()
    df_test_temp = df_test.copy()

    for col in ['Sex', 'Cabin', 'Embarked']:

        # make dictionary mapping category to counts
        counts_map = df_train_temp[col].value_counts().to_dict()

        # remap the labels to their counts
        df_train_temp[col] = df_train_temp[col].map(counts_map)
        df_test_temp[col] = df_test_temp[col].map(counts_map)

    return df_train_temp, df_test_temp


X_train_count, X_test_count = categorical_to_counts(X_train, X_test)

X_train_count.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
352,3,402,15.0,1,1,7.2292,473,108
125,3,402,12.0,1,0,11.2417,473,108
579,3,402,32.0,0,0,7.9250,473,457
424,3,402,18.0,1,1,20.2125,473,457
119,3,219,2.0,4,2,31.2750,473,457


### Ordered Integer Encoding

In [20]:
def categories_to_ordered(df_train, df_test, y_train, y_test):

    # make a temporary copy of the datasets
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['Sex', 'Cabin', 'Embarked']:

        # order categories according to target mean
        ordered_labels = df_train_temp.groupby(
            [col])['Survived'].mean().sort_values().index

        # create the dictionary to map the ordered labels to an ordinal number
        ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}

        # remap the categories  to these ordinal numbers
        df_train_temp[col] = df_train[col].map(ordinal_label)
        df_test_temp[col] = df_test[col].map(ordinal_label)

    # remove the target
    df_train_temp.drop(['Survived'], axis=1, inplace=True)
    df_test_temp.drop(['Survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_ordered, X_test_ordered = categories_to_ordered(
    X_train, X_test, y_train, y_test)

X_train_ordered.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
352,3,0,15.0,1,1,7.2292,0,2
125,3,0,12.0,1,0,11.2417,0,2
579,3,0,32.0,0,0,7.9250,0,0
424,3,0,18.0,1,1,20.2125,0,0
119,3,1,2.0,4,2,31.2750,0,0


### Mean Encoding

In [21]:
def categories_to_mean(df_train, df_test, y_train, y_test):

    # make a temporary copy of the datasets
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['Sex', 'Cabin', 'Embarked']:

        # calculate mean target per category
        ordered_labels = df_train_temp.groupby(
            [col])['Survived'].mean().to_dict()

        # remap the categories to target mean
        df_train_temp[col] = df_train[col].map(ordered_labels)
        df_test_temp[col] = df_test[col].map(ordered_labels)

    # remove the target
    df_train_temp.drop(['Survived'], axis=1, inplace=True)
    df_test_temp.drop(['Survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_mean, X_test_mean = categories_to_mean(
    X_train, X_test, y_train, y_test)

X_train_mean.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
352,3,0.199005,15.0,1,1,7.2292,0.308668,0.555556
125,3,0.199005,12.0,1,0,11.2417,0.308668,0.555556
579,3,0.199005,32.0,0,0,7.9250,0.308668,0.347921
424,3,0.199005,18.0,1,1,20.2125,0.308668,0.347921
119,3,0.739726,2.0,4,2,31.2750,0.308668,0.347921


### WoE

In [27]:
def categories_to_woe(df_train, df_test, y_train, y_test):
    # make a temporary copy of the datasets
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()
    
    #print(df_train_temp['Survived'].mean())
    for col in ['Sex', 'Cabin', 'Embarked']:
        # create df containing the different parts of the WoE equation
        # prob survived =1
        prob_df = pd.DataFrame(df_train_temp.groupby([col])['Survived'].mean())
        # prob survived = 0
        prob_df['died'] = 1-prob_df.Survived
        # calculate WoE
        prob_df['WoE'] = np.log(prob_df.Survived/prob_df.died)
        # capture woe in dictionary
        woe = prob_df['WoE'].to_dict()
        # re-map the labels to WoE
        df_train_temp[col] = df_train[col].map(woe)
        df_test_temp[col] = df_test[col].map(woe)
    # drop the target
    df_train_temp.drop(['Survived'], axis=1, inplace=True)
    df_test_temp.drop(['Survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_woe, X_test_woe = categories_to_woe(X_train, X_test, y_train, y_test)

X_train_woe.head()

Sex
Cabin
Embarked


,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
352,3,-1.392525,15.0,1,1,7.2292,-0.806354,0.223144
125,3,-1.392525,12.0,1,0,11.2417,-0.806354,0.223144
579,3,-1.392525,32.0,0,0,7.9250,-0.806354,-0.628189
424,3,-1.392525,18.0,1,1,20.2125,-0.806354,-0.628189
119,3,1.044545,2.0,4,2,31.2750,-0.806354,-0.628189


### BackwardDifferenceEncoder

In [0]:

encoder = ce.BackwardDifferenceEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_backdiffencoded=encoder.transform(X_train)
X_test_backdiffencoded=encoder.transform(X_test)
X_train_backdiffencoded.head()

,intercept,Pclass,Sex_0,Age,SibSp,Parch,Fare,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Cabin_4,Cabin_5,Cabin_6,Embarked_0,Embarked_1
352,1,3,-0.5,15.0,1,1,7.2292,-0.875,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,-0.666667,-0.333333
125,1,3,-0.5,12.0,1,0,11.2417,-0.875,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,-0.666667,-0.333333
579,1,3,-0.5,32.0,0,0,7.9250,-0.875,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,0.333333,-0.333333
424,1,3,-0.5,18.0,1,1,20.2125,-0.875,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,0.333333,-0.333333
119,1,3,0.5,2.0,4,2,31.2750,-0.875,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,0.333333,-0.333333


### BaseN

In [29]:

encoder = ce.BaseNEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_BaseN=encoder.transform(X_train)
X_test_BaseN=encoder.transform(X_test)
X_train_BaseN.head()

,Pclass,Sex_0,Sex_1,Age,SibSp,Parch,Fare,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Embarked_0,Embarked_1,Embarked_2
352,3,0,1,15.0,1,1,7.2292,0,0,0,1,0,0,1
125,3,0,1,12.0,1,0,11.2417,0,0,0,1,0,0,1
579,3,0,1,32.0,0,0,7.9250,0,0,0,1,0,1,0
424,3,0,1,18.0,1,1,20.2125,0,0,0,1,0,1,0
119,3,1,0,2.0,4,2,31.2750,0,0,0,1,0,1,0


### Binary 

In [37]:

encoder = ce.BinaryEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_Binary=encoder.transform(X_train)
X_test_Binary=encoder.transform(X_test)
X_train_Binary.head()

,Pclass,Sex_0,Sex_1,Age,SibSp,Parch,Fare,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Embarked_0,Embarked_1,Embarked_2
352,3,0,1,15.0,1,1,7.2292,0,0,0,1,0,0,1
125,3,0,1,12.0,1,0,11.2417,0,0,0,1,0,0,1
579,3,0,1,32.0,0,0,7.9250,0,0,0,1,0,1,0
424,3,0,1,18.0,1,1,20.2125,0,0,0,1,0,1,0
119,3,1,0,2.0,4,2,31.2750,0,0,0,1,0,1,0


### CatBoost Encoder

In [40]:

encoder = ce.CatBoostEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_CatBoost=encoder.transform(X_train)
X_test_CatBoost=encoder.transform(X_test)
X_train_CatBoost.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
352,3,0.199478,15.0,1,1,7.2292,0.308839,0.554034
125,3,0.199478,12.0,1,0,11.2417,0.308839,0.554034
579,3,0.199478,32.0,0,0,7.9250,0.308839,0.348012
424,3,0.199478,18.0,1,1,20.2125,0.308839,0.348012
119,3,0.738135,2.0,4,2,31.2750,0.308839,0.348012


### Hashing Encoding

In [43]:

encoder = ce.HashingEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_Hashed=encoder.transform(X_train)
X_test_Hashed=encoder.transform(X_test)
X_train_Hashed.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,Pclass,Age,SibSp,Parch,Fare
352,0,1,0,0,0,1,0,1,3,15.0,1,1,7.2292
125,0,1,0,0,0,1,0,1,3,12.0,1,0,11.2417
579,0,1,0,0,0,1,1,0,3,32.0,0,0,7.9250
424,0,1,0,0,0,1,1,0,3,18.0,1,1,20.2125
119,0,1,0,0,0,1,1,0,3,2.0,4,2,31.2750


### Helmert Coding

In [46]:

encoder = ce.HelmertEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_HelmertEncoded=encoder.transform(X_train)
X_test_HelmertEncoded=encoder.transform(X_test)
X_train_HelmertEncoded.head()

,intercept,Pclass,Sex_0,Age,SibSp,Parch,Fare,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Cabin_4,Cabin_5,Cabin_6,Embarked_0,Embarked_1
352,1,3,-1.0,15.0,1,1,7.2292,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
125,1,3,-1.0,12.0,1,0,11.2417,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
579,1,3,-1.0,32.0,0,0,7.9250,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
424,1,3,-1.0,18.0,1,1,20.2125,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
119,1,3,1.0,2.0,4,2,31.2750,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0


### James-Stein Encoding

In [49]:

encoder = ce.JamesSteinEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_JamesSteinEncoded=encoder.transform(X_train)
X_test_JamesSteinEncoded=encoder.transform(X_test)
X_train_JamesSteinEncoded.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
352,3,0.199005,15.0,1,1,7.2292,0.336046,0.555556
125,3,0.199005,12.0,1,0,11.2417,0.336046,0.555556
579,3,0.199005,32.0,0,0,7.9250,0.336046,0.347921
424,3,0.199005,18.0,1,1,20.2125,0.336046,0.347921
119,3,0.739726,2.0,4,2,31.2750,0.336046,0.347921


### Leave One Out

In [52]:

encoder = ce.LeaveOneOutEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_LeaveOneOutEncoded=encoder.transform(X_train)
X_test_LeaveOneOutEncoded=encoder.transform(X_test)
X_train_LeaveOneOutEncoded.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
352,3,0.199005,15.0,1,1,7.2292,0.308668,0.555556
125,3,0.199005,12.0,1,0,11.2417,0.308668,0.555556
579,3,0.199005,32.0,0,0,7.9250,0.308668,0.347921
424,3,0.199005,18.0,1,1,20.2125,0.308668,0.347921
119,3,0.739726,2.0,4,2,31.2750,0.308668,0.347921


### M-estimate

In [55]:

encoder = ce.MEstimateEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_MEstimateEncoded=encoder.transform(X_train)
X_test_MEstimateEncoded=encoder.transform(X_test)
X_train_MEstimateEncoded.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
352,3,0.199478,15.0,1,1,7.2292,0.308839,0.554034
125,3,0.199478,12.0,1,0,11.2417,0.308839,0.554034
579,3,0.199478,32.0,0,0,7.9250,0.308839,0.348012
424,3,0.199478,18.0,1,1,20.2125,0.308839,0.348012
119,3,0.738135,2.0,4,2,31.2750,0.308839,0.348012


### Polynomial Coding

In [56]:

encoder = ce.PolynomialEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_PolynomialEncoded=encoder.transform(X_train)
X_test_PolynomialEncoded=encoder.transform(X_test)
X_train_PolynomialEncoded.head()

,intercept,Pclass,Sex_0,Age,SibSp,Parch,Fare,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Cabin_4,Cabin_5,Cabin_6,Embarked_0,Embarked_1
352,1,3,-0.707107,15.0,1,1,7.2292,-0.540062,0.540062,-0.43082,0.282038,-0.149786,0.061546,-0.01707,-7.071068e-01,0.408248
125,1,3,-0.707107,12.0,1,0,11.2417,-0.540062,0.540062,-0.43082,0.282038,-0.149786,0.061546,-0.01707,-7.071068e-01,0.408248
579,1,3,-0.707107,32.0,0,0,7.9250,-0.540062,0.540062,-0.43082,0.282038,-0.149786,0.061546,-0.01707,-5.551115e-17,-0.816497
424,1,3,-0.707107,18.0,1,1,20.2125,-0.540062,0.540062,-0.43082,0.282038,-0.149786,0.061546,-0.01707,-5.551115e-17,-0.816497
119,1,3,0.707107,2.0,4,2,31.2750,-0.540062,0.540062,-0.43082,0.282038,-0.149786,0.061546,-0.01707,-5.551115e-17,-0.816497


### Sum Coding

In [57]:

encoder = ce.SumEncoder(cols=['Sex','Cabin','Embarked'])
encoder.fit(X_train,y_train)
X_train_SumEncoded=encoder.transform(X_train)
X_test_SumEncoded=encoder.transform(X_test)
X_train_SumEncoded.head()

,intercept,Pclass,Sex_0,Age,SibSp,Parch,Fare,Cabin_0,Cabin_1,Cabin_2,Cabin_3,Cabin_4,Cabin_5,Cabin_6,Embarked_0,Embarked_1
352,1,3,1.0,15.0,1,1,7.2292,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
125,1,3,1.0,12.0,1,0,11.2417,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
579,1,3,1.0,32.0,0,0,7.9250,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
424,1,3,1.0,18.0,1,1,20.2125,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
119,1,3,-1.0,2.0,4,2,31.2750,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Random Forest Performance

In [0]:
# create a function to build random forests and compare performance in train and test set


def run_randomForests(X_train, X_test, y_train, y_test):

    rf = RandomForestClassifier(n_estimators=50, random_state=39, max_depth=3)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict_proba(X_train)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Test set')
    pred = rf.predict_proba(X_test)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [0]:
# OHE
run_randomForests(X_train_OHE, X_test_OHE, y_train, y_test)

Train set
Random Forests roc-auc: 0.8604254344839617
Test set
Random Forests roc-auc: 0.8677394034536892


In [0]:
# counts
run_randomForests(X_train_count, X_test_count, y_train, y_test)

Train set
Random Forests roc-auc: 0.8760494123290956
Test set
Random Forests roc-auc: 0.8819587006400194


In [0]:
# ordered labels
run_randomForests(X_train_ordered, X_test_ordered, y_train, y_test)

Train set
Random Forests roc-auc: 0.8770960989118821
Test set
Random Forests roc-auc: 0.8806605482429659


In [0]:
# mean encoding
run_randomForests(X_train_mean, X_test_mean, y_train, y_test)

Train set
Random Forests roc-auc: 0.8770960989118821
Test set
Random Forests roc-auc: 0.8806605482429659


In [31]:
# woe
run_randomForests(X_train_woe, X_test_woe, y_train, y_test)

Train set
Random Forests roc-auc: 0.8770960989118821
Test set
Random Forests roc-auc: 0.8806605482429659


In [0]:
# Back Diff encoding
run_randomForests(X_train_backdiffencoded, X_test_backdiffencoded, y_train, y_test)

Train set
Random Forests roc-auc: 0.8695839420833423
Test set
Random Forests roc-auc: 0.8808114961961115


In [32]:
# BaseN encoding
run_randomForests(X_train_BaseN, X_test_BaseN, y_train, y_test)


Train set
Random Forests roc-auc: 0.8659914084476329
Test set
Random Forests roc-auc: 0.877490641226905


In [38]:
# Binary encoding
run_randomForests(X_train_Binary, X_test_Binary, y_train, y_test)


Train set
Random Forests roc-auc: 0.8659914084476329
Test set
Random Forests roc-auc: 0.877490641226905


In [41]:
# Catboost encoding
run_randomForests(X_train_CatBoost, X_test_CatBoost, y_train, y_test)


Train set
Random Forests roc-auc: 0.8770960989118821
Test set
Random Forests roc-auc: 0.8806605482429659


In [45]:
# Hashed Encoder
run_randomForests(X_train_Hashed, X_test_Hashed, y_train, y_test)


Train set
Random Forests roc-auc: 0.7754257615735187
Test set
Random Forests roc-auc: 0.762589059292356


In [47]:
# Helmert Encoder
run_randomForests(X_train_HelmertEncoded, X_test_HelmertEncoded, y_train, y_test)


Train set
Random Forests roc-auc: 0.8698619682068949
Test set
Random Forests roc-auc: 0.8791208791208791


In [50]:
# James Stein Encoder
run_randomForests(X_train_JamesSteinEncoded, X_test_JamesSteinEncoded, y_train, y_test)


Train set
Random Forests roc-auc: 0.8770960989118821
Test set
Random Forests roc-auc: 0.8806605482429659


In [53]:
# LeaveOneOutEncoder
run_randomForests(X_train_LeaveOneOutEncoded, X_test_LeaveOneOutEncoded, y_train, y_test)


Train set
Random Forests roc-auc: 0.8770960989118821
Test set
Random Forests roc-auc: 0.8806605482429659


In [58]:
# MEstimateEncoder
run_randomForests(X_train_MEstimateEncoded, X_test_MEstimateEncoded, y_train, y_test)


Train set
Random Forests roc-auc: 0.8770960989118821
Test set
Random Forests roc-auc: 0.8806605482429659


In [60]:
# PolynomialEncoder
run_randomForests(X_train_PolynomialEncoded, X_test_PolynomialEncoded, y_train, y_test)


Train set
Random Forests roc-auc: 0.8694967182014435
Test set
Random Forests roc-auc: 0.8768264702330636


In [63]:
# SumEncoder
run_randomForests(X_train_SumEncoded, X_test_SumEncoded, y_train, y_test)


Train set
Random Forests roc-auc: 0.8753189123181926
Test set
Random Forests roc-auc: 0.8759811616954474


Comparing the roc_auc values on the test sets, we can see that one hot encoding has the worse performance. This makes sense because trees do not perform well in datasets with big feature spaces.

The remaining encodings returned similar performances. This also makes sense, because trees are non-linear models, so target guided encodings may not necessarily improve the model performance

### Logistic Regression Performance

In [0]:
def run_logistic(X_train, X_test, y_train, y_test):

    # function to train and test the performance of logistic regression
    logit = LogisticRegression(random_state=44, C=0.01)
    logit.fit(X_train, y_train)

    print('Train set')
    pred = logit.predict_proba(X_train)
    print(
        'Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Test set')
    pred = logit.predict_proba(X_test)
    print(
        'Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [0]:
# OHE
run_logistic(X_train_OHE, X_test_OHE, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8299952026864956
Test set
Logistic Regression roc-auc: 0.8428329911846394


In [0]:
# counts
run_logistic(X_train_count, X_test_count, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8447578446978783
Test set
Logistic Regression roc-auc: 0.8583504407680232


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
# ordered labels
run_logistic(X_train_ordered, X_test_ordered, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8143385158856495
Test set
Logistic Regression roc-auc: 0.8309382924767539


In [0]:
# mean encoding
run_logistic(X_train_mean, X_test_mean, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.7738339257288646
Test set
Logistic Regression roc-auc: 0.7892766574085255


In [35]:
# woe
run_logistic(X_train_woe, X_test_woe, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8506236507555769
Test set
Logistic Regression roc-auc: 0.8726603067262407


In [0]:
# backdiffencoding
run_logistic(X_train_backdiffencoded, X_test_backdiffencoded, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.826135545912471
Test set
Logistic Regression roc-auc: 0.843919816447289


In [36]:

# Base N Encoder
run_logistic(X_train_BaseN, X_test_BaseN, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8435476133365316
Test set
Logistic Regression roc-auc: 0.8610675039246467


In [39]:
# Binary Encoder
run_logistic(X_train_Binary, X_test_Binary, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8435476133365316
Test set
Logistic Regression roc-auc: 0.8610675039246467


In [42]:
# CatBoost Encoder
run_logistic(X_train_CatBoost, X_test_CatBoost, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.7734959331865066
Test set
Logistic Regression roc-auc: 0.7889143823209757


In [44]:
# Hashed Encoder
run_logistic(X_train_Hashed, X_test_Hashed, y_train, y_test)


Train set
Logistic Regression roc-auc: 0.7374561154844196
Test set
Logistic Regression roc-auc: 0.733607052288371


In [48]:
# Helmert Encoder
run_logistic(X_train_HelmertEncoded, X_test_HelmertEncoded, y_train, y_test)


Train set
Logistic Regression roc-auc: 0.8505255238884407
Test set
Logistic Regression roc-auc: 0.866139355150344


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [51]:
# James Stein Encoder
run_logistic(X_train_JamesSteinEncoded, X_test_JamesSteinEncoded, y_train, y_test)


Train set
Logistic Regression roc-auc: 0.7734959331865064
Test set
Logistic Regression roc-auc: 0.7893974157710422


In [54]:

# LeaveOneOutEncoder
run_logistic(X_train_LeaveOneOutEncoded, X_test_LeaveOneOutEncoded, y_train, y_test)


Train set
Logistic Regression roc-auc: 0.7738339257288646
Test set
Logistic Regression roc-auc: 0.7892766574085255


In [59]:
# MEstimateEncoder
run_logistic(X_train_MEstimateEncoded, X_test_MEstimateEncoded, y_train, y_test)


Train set
Logistic Regression roc-auc: 0.7734959331865066
Test set
Logistic Regression roc-auc: 0.7889143823209757


In [61]:
# PolynomialEncoder
run_logistic(X_train_PolynomialEncoded, X_test_PolynomialEncoded, y_train, y_test)


Train set
Logistic Regression roc-auc: 0.8447033297716915
Test set
Logistic Regression roc-auc: 0.8603429537495472


In [62]:
# SumEncoder
run_logistic(X_train_SumEncoded, X_test_SumEncoded, y_train, y_test)


Train set
Logistic Regression roc-auc: 0.8487265313242767
Test set
Logistic Regression roc-auc: 0.869279072575776


For Logistic regression, the best performances are obtained with one hot encoding, as it preserves linear relationships with variables and target, and also with weight of evidence, and ordered encoding.

Note however how count encoding, returns the worse performance as it does not create a monotonic relationship between variables and target, and in this case, mean target encoding is probably causing over-fitting.